In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pyxu.abc import DiffFunc, LinOp
import pyxu.operator as pxop

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

### Import neuron data

In [35]:
# Read neuron spike data
data = pd.read_csv("simulated_data/events0.csv", header=None)
data = [data.iloc[i].values for i in range(len(data))]  # convert to list of numpy arrays

In [36]:
# Obtain useful constants
M = len(data)  # number of neurons

t = data  # pour faciliter les notations apres
for i in range(M):
    t[i] = t[i][np.nonzero(t[i])]  # get rid of arrival times = 0

k = [len(ti) for ti in t]  # number of arrivals of each neuron
T = max(max(t[i]) for i in range(M))  # TODO: Do i need T to be the last arrival, or > last arrival ?
K = sum(k)

In [13]:
print(k, 'number of arrivals of each neuron')
print(M, '= M, total number of neurons')
print(len(k), 'should be equal to M')
print(K, 'total number of arrivals')

[100, 165, 244, 116, 255, 342, 226, 353] number of arrivals of each neuron
8 = M, total number of neurons
8 should be equal to M
1801 total number of arrivals


In [8]:
# Plot spike train for all neurons
plotspike, ax = plt.subplots(1, 1, figsize= (10,8))
for i in range(M):
    ax.plot(t[i], np.zeros(len(t[i]))+i, linestyle='', marker='+')
ax.set_xlabel('Spike trains')
ax.set_ylabel('Neurons')

Text(0, 0.5, 'Neurons')

### Compute A (discrete likelihood matrix)

In [39]:
# Setup the HP
beta = 1  # we choose this
betainv = 1/beta
def g(t, beta=beta):
    return t * np.exp(-beta*t) * (t >= 0)

In [10]:
# Compute A^j for j=1,...,M
A = [np.zeros((k[j]+1, M+1)) for j in range(M)]  # all matrices A

B = np.zeros(M)  # to help compute the last row of A^j
for n in range(M):
    B[n] = betainv * sum((T - tnl + betainv)*np.exp(-beta*(T - tnl)) - betainv for tnl in t[n])

# Fill out A^j for every j
for j in range(M):
    # First column computation
    A[j][:, 0] = 1
    A[j][-1, 0] = -T

    # Fill out the rest of the matrix (-1 because last row is filled out separately)
    for i in range(A[j].shape[0] - 1):
        for n in range(M):
            A[j][i, n+1] = sum(g(t[j][i] - tnl) for tnl in t[n] if tnl < t[j][i])

    # Fill out last row
    for n in range(M):
        A[j][-1, n+1] = B[n]

In [41]:
print(sum((T - tnl + betainv)*np.exp(-beta*(T - tnl)) - betainv for tnl in t[0]))

0.9107304665138942


In [48]:
# Plot matrix A (without last row, to keep things to scale)
j = 0

fig, ax = plt.subplots(1, 1, figsize=(5,8))
im = ax.imshow(A[j][0:-1, :], cmap='viridis', interpolation='nearest', extent=[1, M, 0, k[j]/5])
fig.colorbar(im, ax=ax)  # Add a colorbar to show the scale
ax.set_title('Matrix Coefficients')
plt.show()

### Define -log(likelihood) operator

<font size="3"> Recall that we want to minimize $-\ln(L(\theta)) = E(A\theta)$ where: 
- $E: \mathbb{R}^{M+K} \to \mathbb{R}$, $x \mapsto E(x) = -\sum_{x \not\in S} \ln(x_i) + \sum_{x \in S} x_i$, $S \subset \{1, ..., M+K\}$ the set of indices contributing to the compensator term
- $A \in \mathbb{R}^{(M+K) \times (M + M^2)}$ is the discrete likelihood matrix
- $\theta \in \mathbb{R}^{M + M^2}$ is the vector of parameters, $\theta = (\mu_1, \vec{\alpha}_1, ..., \mu_M, \vec{\alpha}_M)$ 
</font>

In [14]:
# # Toy dimensions
# M = 1  # total number of processes
# K = 10  # total number of arrivals across all processes

# # Define random matrix A of size (M+K) x M
# A = np.random.normal(0, 1, (K+M, M*(1+M)))

In [30]:
# Define each A^j as an operator
ops = [None for _ in range(M)]

for i in range(M):
    ops[i] = LinOp.from_array(A[i])

# Define A of shape (K+M, M*(1+M)) as block-diagonal. TODO: maybe implement this in a matrix free way ?
opA = pxop.block_diag(ops)

In [18]:
# Define subset S of {1, ..., M+K} of row indices of A which contribute to the compensator term
S = np.zeros(M)  # we know that |S| = M
for i in range(M):
    S[i] = sum(k[:(i+1)]) + i + 1
S = [int(si)-1 for si in S]  # convert to int, decaler de -1 pour avoir tout entre 0 et M+K-1

# Define the complement of S
allIndices = list(range(0, M+K))  # or range(1, M+K+1)
notS = list(set(allIndices) - set(S))

In [19]:
# Define E: R^{K+M} -> R, E(x) = -\sum_{i \not\in S} ln(x_i) + \sum_{i \in S} as a DiffFunc
class LikelihoodE(DiffFunc):
    def __init__(self, dim):
        super().__init__(shape=(1, dim))

    def apply(self, arr):
        return sum(-np.log(arr[notS])) + sum(arr[S])

    def grad(self, arr):
        grad = np.ones(arr.shape)
        grad[notS] = -1/arr[notS]
        return grad

In [21]:
dim = M+K
E = LikelihoodE(dim=dim)

x = np.ones(dim)
print(E(x))
print(E.grad(x))

8.0
[-1. -1. -1. ... -1. -1.  1.]


In [34]:
# Define -log(likelihood) (to minimize) as -log(L(theta)) = E(A*theta)
L = E * opA

In [47]:
theta = np.ones(M*(1+M))
print('likelihood:', L(theta))
#print('gradient:', L.grad(theta))
#print('gradient computed by hand:', opA.T(E.grad(opA(theta))))  # true gradient

likelihood: -20217.70024678208


### Test with everything in a class

In [4]:
from hawkes_likelihood import HawkesLikelihood

In [5]:
path = "simulated_data/events0.csv"
hpL = HawkesLikelihood(path=path)

In [6]:
hpL.plot_realization()

In [7]:
j = 0
hpL.plot_A(idx=j)

In [21]:
print(hpL.opA.shape)  # = (K+M, M^2 + M)
print(hpL.negLogL.shape)

(1809, 72)
(1, 72)


In [8]:
dim = hpL.M + hpL.K  # = hpL.opA.shape[0]

x = np.ones(dim)
print(hpL.E(x))
print(hpL.E.grad(x))

8.0
[-1. -1. -1. ... -1. -1.  1.]


In [23]:
theta = np.ones(hpL.negLogL.shape[1])  # = M^2 + M
print('likelihood:', hpL.negLogL(theta))

likelihood: -20217.70024678208


In [31]:
print(np.where(hpL.A[0] < 0)[0])
print(hpL.A[0])

[100 100 100 100 100 100 100 100 100]
[[ 1.00000000e+00  0.00000000e+00  3.83718731e-02  6.89868199e-01
   2.50440502e+00  5.50751020e-02  4.04780415e+00  3.01104586e+00
   7.89244966e-01]
 [ 1.00000000e+00  1.64940232e-01  5.03086834e-02  6.35130280e-01
   2.43094772e+00  4.64568164e-02  5.14312220e+00  2.63671297e+00
   1.08357986e+00]
 [ 1.00000000e+00  2.26013731e-01  8.49514343e-02  6.24241447e-01
   2.43744738e+00  4.49683575e-02  5.39467806e+00  2.56945051e+00
   1.21168115e+00]
 [ 1.00000000e+00  3.05659160e-01  1.12241820e-01  6.14932637e-01
   2.43984843e+00  4.37412056e-02  5.63736907e+00  2.54388804e+00
   1.31688194e+00]
 [ 1.00000000e+00  1.13447838e+00  3.09553016e-01  8.71396218e-01
   2.66777910e+00  3.21579036e-02  8.24731515e+00  2.36120847e+00
   2.02835953e+00]
 [ 1.00000000e+00  1.20988725e+00  3.19796185e-01  9.32384853e-01
   2.77583952e+00  3.12760683e-02  8.41911417e+00  2.38302877e+00
   2.06021138e+00]
 [ 1.00000000e+00  2.14372868e+00  3.73206739e-01  2.381